# GEN AI Workflow Notebook ✨✨

ClassifAI allows users to create vector databases from text datasets, search those vector datasets in an ad-hoc retrieval manner, and deploy this pipeline as a restAPI service using FastAPI to perform AI assisted classification tasks.


An recent emergent AI field is Retrieval Augmented Generation (RAG) where text generation models that traditionally respond to a user 'prompt', first retrieve relvant infomration from a vector database via adhoc retrieval processes, and use those serach results as context to generate an answer for the original user prompt.

#### This notebook shows how RAG can be implemented into our ClassifAI utilising its existing retrieval augmentation capabilities


## ClassifAIs existing retrieval setup

![Server_Image](files/servers.png)

#### The other modules of ClassifAI provide 3 core classes that work together to:

1. Vectorisers, to create embeddings,
2. Vectorstores, to create datgabsees of vectors and the ability to searc/query those database
3. RestAPI, to deploy a rest api service on a server to allow connections that search the created vector databases

In [ ]:
!uv pip install "classifai[gcp]"

## Setup

We need to first set up a traditional ClassifAI pipeline that can provide search/classification results for our generative model to use as context...

In [ ]:
!gcloud auth application-default login

In [ ]:
from classifai.vectorisers import GcpVectoriser
from classifai.indexers import VectorStore

vectoriser = GcpVectoriser(
    project_id="<GCLOUD PROJECT_ID>",
    model_name="text-embedding-004",
    task_type="CLASSIFICATION",
    )



my_vector_store = VectorStore(
    file_name="<PATH TO CSV FILE>",
    data_type="csv",
    vectoriser=vectoriser,
    overwrite=True,
)

In [ ]:
my_vector_store.search("magic users in onessia", n_results=5)

## Creating a RAG Agent

With a way of getting context information for a query/prompt we can now instantiate a generative model


![Rag_Image](files/agent.png) 


In essence, the RAG agent has a .generate method which accepts a string query. It internally calls the associated vectorstore's search method using that query provided by the user. The original user query and the vectorstore results are used to generate a response.




To instantiate the object it takes:

In [ ]:

from classifai.generators import RagAgent_GCP


my_agent = RagAgent_GCP(
    project_id="<GCLOUD PROJECT_ID>",
    model_name="gemini-2.5-flash",
    vectorStore=my_vector_store,
    task_type="classification",

    )

#### Similar to the Vectorsiser it has a transform method -

#### this executes a vector store search, then uses that ranking and the user query to generate a response in line with the TASK_TYPE

In [ ]:
my_agent.transform("magic users in onessia?")